Les réseaux de neurones convolutifs ont été utilisés pour reconnaître les chiffres des chèques à la fin des années 90. Il y a eu une base solide pendant tout ce temps, alors pourquoi a-t-on l'impression qu'une explosion s'est produite au cours des dix dernières années ?

Les raisons sont nombreuses, mais la principale d'entre elles est l'augmentation des performances des GPU et leur prix de plus en plus abordable. Conçus à l'origine pour les jeux, les GPU doivent effectuer des millions d'opérations matricielles par seconde afin de restituer tous les polygones du jeu de conduite ou de tir auquel vous jouez sur votre console ou votre PC, opérations pour lesquelles un CPU standard n'est tout simplement pas optimisé.

Un article publié en 2009~[[1]](#1) soulignait que la formation des réseaux de neurones reposait également sur l'exécution d'un grand nombre d'opérations matricielles. Ces cartes graphiques supplémentaires pourraient donc être utilisées pour accélérer la formation et rendre possible, pour la première fois des architectures de réseaux neuronaux plus grandes et plus profondes.

# Démarrer avec PyTorch

In [1]:
!nvidia-smi

Thu Jul  1 19:39:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:2D:00.0  On |                  N/A |
|  0%   43C    P5    39W / 215W |   1065MiB /  7981MiB |     35%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

CUDA Version 11.3, on peut donc procéder à l'installation de PyTorch via les instructions trouvable sur leur [site web](https://pytorch.org/).

In [2]:
!pip3 install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio==0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/cu111/torch-1.9.0%2Bcu111-cp38-cp38-linux_x86_64.whl (2041.3 MB)
  Using cached https://download.pytorch.org/whl/cu111/torchvision-0.10.0%2Bcu111-cp38-cp38-linux_x86_64.whl (23.2 MB)
  Using cached torchaudio-0.9.0-cp38-cp38-manylinux1_x86_64.whl (1.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.8.1
    Uninstalling torch-1.8.1:
      Successfully uninstalled torch-1.8.1


On s'assure ici que CUDA est accesible et crée un tenseur à valeurs aléatoires de taille 2*2

In [3]:
import torch
print(torch.cuda.is_available())
print(torch.rand(2, 2))

True
tensor([[0.0013, 0.9979],
        [0.3422, 0.2302]])


## Tenseurs

Un tenseur est à la fois un conteneur pour les nombres et un ensemble de tuples qui définissent les transformations entre les tenseurs qui produisent de nouveaux tenseurs.

Chaque tenseur a un rang qui correspond à son espace dimensionnel. Un simple scalaire—par exemple, 1—peut être représenté comme un tenseur de rang 0, un vecteur est de rang 1, une matrice $n*n$ est de rang 2, et ainsi de suite. Dans l'exemple précédent, nous avons créé un tenseur de rang 2 avec des valeurs aléatoires en utilisant `torch.rand()`. Nous pouvons également les créer à partir de listes :

In [7]:
x = torch.tensor([[0, 0, 1], [1, 1, 1], [0, 0, 0]])
x

tensor([[0, 0, 1],
        [1, 1, 1],
        [0, 0, 0]])

On peut changer les éléments d'un tenseur en utilisant le système standard d'indexation de Python

In [8]:
x[0][0] = 5
x

tensor([[5, 0, 1],
        [1, 1, 1],
        [0, 0, 0]])

On peut utiliser des fonctions de création spéciales pour générer des types particuliers de tenseurs. En particulier, `ones()` et `zeros()` vont générer des tenseurs remplis de 1 et de 0, respectivement :

In [9]:
torch.zeros(2, 2)

tensor([[0., 0.],
        [0., 0.]])

On peut effectuer des opérations mathématiques standard avec des tenseurs (par exemple, additionner deux tenseurs) :

In [13]:
torch.ones(1, 2) + torch.ones(1, 2)

tensor([[2., 2.]])

Et si vous avez un tenseur de rang 0, vous pouvez en extraire la valeur avec `item()` :

In [14]:
torch.rand(1).item()

0.013902544975280762

Les tenseurs peuvent vivre dans le CPU ou sur le GPU et peuvent être copiés entre les dispositifs en utilisant la fonction `to()` :

In [16]:
cpu_tensor = torch.rand(2)
cpu_tensor.device

device(type='cpu')

In [17]:
gpu_tensor = cpu_tensor.to("cuda")
gpu_tensor.device

device(type='cuda', index=0)

## Opérations Tensorielles

Tout d'abord, nous avons souvent besoin de trouver l'élément maximal dans un tenseur ainsi que l'index qui contient la valeur maximale. Ceci peut être fait avec les fonctions `max()` et `argmax()`. Nous pouvons également utiliser `item()` pour extraire une valeur standard Python d'un tenseur à une dimension.

In [18]:
torch.rand(2, 2).max()

tensor(0.3832)

In [19]:
torch.rand(2, 2).max().item()

0.950043261051178

Parfois, nous souhaitons changer le type d'un tenseur, par exemple en passant d'un **LongTensor** à un **FloatTensor**. Nous pouvons le faire avec `to()` :

In [20]:
long_tensor = torch.tensor([[0, 0, 1], [1, 1, 1], [0, 0, 0]])
long_tensor.type()

'torch.LongTensor'

In [21]:
float_tensor = torch.tensor([[0, 0, 1], [1, 1, 1], [0, 0, 0]]).to(dtype=torch.float32)
float_tensor.type()

'torch.FloatTensor'

La plupart des fonctions qui opèrent sur un tenseur et retournent un tenseur créent un nouveau tenseur pour stocker le résultat. Toutefois, si vous voulez économiser de la mémoire, vérifiez si une fonction *in-place* est définie, elle devrait avoir le même nom que la fonction originale mais avec un underscore (_).

In [22]:
random_tensor = torch.rand(2, 2)
random_tensor.log2()

tensor([[-2.3971, -0.6222],
        [-1.7551, -0.2403]])

In [23]:
random_tensor.log2_()

tensor([[-2.3971, -0.6222],
        [-1.7551, -0.2403]])

Une autre opération courante consiste à remodeler un tenseur. Cela peut souvent se produire parce que la couche de votre réseau de neurones peut nécessiter une forme d'entrée légèrement différente de celle que vous avez actuellement à lui fournir. Par exemple, l'ensemble de données de *Modified National Institute of Standard and Technology* (MNIST) de chiffres manuscrits est une collection de $28*28$ images, mais il est présenté sous forme de tableaux de tenseurs de longueur $1*28*28$ (le 1 initial correspond au nombre de canaux - normalement rouge, vert et bleu - mais comme les chiffres MNIST sont en niveaux de gris, nous n'avons qu'un seul canal). Nous pouvons faire cela avec `view()` ou `reshape()` :

In [25]:
flat_tensor = torch.rand(784)
viewed_tensor = flat_tensor.view(1, 28, 28)
viewed_tensor.shape

torch.Size([1, 28, 28])

In [26]:
reshaped_tensor = flat_tensor.reshape(1, 28, 28)
reshaped_tensor.shape

torch.Size([1, 28, 28])

Maintenant, vous vous demandez peut-être quelle est la différence entre `view()` et `reshape()`. La réponse est que `view()` opère comme une vue sur le tenseur original, donc si les données sous-jacentes sont modifiées, la vue le sera aussi (et vice versa). Cependant, `view()` peut provoquer des erreurs si la vue requise n'est pas [contiguous](https://stackoverflow.com/questions/26998223/what-is-the-difference-between-contiguous-and-non-contiguous-arrays/26999092#26999092) ; c'est-à-dire qu'elle ne partage pas le même bloc de mémoire qu'elle occuperait si un nouveau tenseur de la forme requise était créé de toutes pièces. Si cela se produit, vous devez appeler `tensor.contiguous()` avant de pouvoir utiliser `view()`.

Enfin, vous pouvez avoir besoin de réorganiser les dimensions d'un tenseur. Vous rencontrerez probablement ce problème avec les images, qui sont souvent stockées sous forme de tenseurs \[height, width, channel\], mais PyTorch préfère les traiter dans un \[channel, height, width\]. Vous pouvez utiliser `permute()` pour les traiter de manière assez simple :

In [27]:
hwc_tensor = torch.rand(640, 480, 3)
chw_tensor = hwc_tensor.permute(2, 0, 1)
chw_tensor.shape

torch.Size([3, 640, 480])

# Classification d'Images avec PyTorch

## Défintion du Problème

Ici, nous construisons un classificateur simple qui peut faire la différence entre les poissons et les chats. Nous allons itérer sur la conception et la manière dont nous construisons notre modèle pour le rendre plus précis.

## Challenges Habituels

Premièrement, nous avons besoin de données. Combien de données ? ça dépend. L'idée selon laquelle, pour que toute technique d'apprentissage profond fonctionne, il faut de grandes quantités de données pour entraîner le réseau de neurones n'est pas nécessairement vraie. Cependant, pour l'instant, nous allons nous entraîner à partir de zéro, ce qui nécessite souvent l'accès à une grande quantité de données. Nous avons besoin de beaucoup de photos de poissons et de chats.

Nous pourrions passer du temps à télécharger de nombreuses images à partir d'un moteur de recherche comme Google Image, mais dans ce cas, nous disposons d'un raccourci : une collection standard d'images utilisée pour former les réseaux neuronaux, appelée *ImageNet*. Elle contient plus de 14 millions d'images et 20 000 catégories d'images. C'est la norme à laquelle tous les classificateurs d'images se mesurent.

Comme nous utilisons les données ImageNet, leurs étiquettes ne seront pas très utiles, car elles contiennent trop d'informations pour nous. Une étiquette de chat tabby ou de truite est, pour l'ordinateur, distincte de celle de chat ou de poisson. Nous devrons les réétiqueter.

## PyTorch et le Chargement de Données

Le chargement et la conversion des données dans des formats prêts pour l'entraînement peuvent souvent finir par être l'un des domaines de la science des données qui accapare beaucoup trop de notre temps (comme pour ce foutu NER pour WaKED). PyTorch a développé des conventions standard d'interaction avec les données qui rendent le travail assez cohérent, que vous travailliez avec des images, du texte ou de l'audio.

Les deux principales conventions d'interaction avec les données sont les *datasets* et les *data loaders*. Un *datasets* est une classe Python qui nous permet d'accéder aux données que nous fournissons au réseau neuronal. Un *data loaders* est ce qui alimente le réseau en données à partir du *datasets*.

Examinons d'abord le *dataste*. Tout ensemble de données, qu'il comprenne des images, de l'audio, du texte, des paysages en 3D, des informations boursières, ou autre, peut interagir avec PyTorch s'il satisfait à cette classe Python abstraite :

In [1]:
class Dataset(object):
    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

Nous devons implémenter une méthode qui renvoie la taille de notre jeu de données (`len`), et implémenter une méthode qui peut récupérer un élément de notre jeu de données dans une paire (**étiquette**, **tenseur**). Cette méthode est appelée par le *data loader* lorsqu'il introduit des données dans le réseau neuronal pour la formation. Nous devons donc écrire un corps pour `getitem` qui peut prendre l'image et la transformer en un tenseur et le retourner avec le label pour que PyTorch puisse l'utiliser.

## Construire des Données d'Entraînement

Le paquet torchvision comprend une classe appelée ImageFolder qui fait à peu près tout pour nous, à condition que nos images soient dans une structure où chaque répertoire est une étiquette.

In [ ]:
import torchvision
from torchvision import transforms

train_data_path = "./train/"

transforms = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_data = torchvision.datasets.ImageFolder(
    root=train_data_path, transform=transforms
)

Les GPU sont conçus pour être rapides dans l'exécution de calculs de taille standard. Mais nous avons probablement un assortiment d'images à de nombreuses résolutions. Pour augmenter nos performances de traitement, nous mettons à l'échelle chaque image entrante à la même résolution de 64*64 via la transformation `Resize(64)`. Nous convertissons ensuite les images en un tenseur, et enfin, nous normalisons le tenseur autour d'un ensemble spécifique de points de moyenne et de déviation standard.

La normalisation est importante car de nombreuses multiplications se produiront lorsque l'entrée passera par les couches du réseau de neurones ; le fait de maintenir les valeurs entrantes entre 0 et 1 empêche les valeurs de devenir trop grandes pendant la phase d'entraînement (connu sous le nom de [*problème d'explosion du gradient*](https://machinelearningmastery.com/exploding-gradients-in-neural-networks/)). 

La moyenne et l'écart-type choisis sont ceux de l'ensemble de données ImageNet dans son ensemble. Vous pourriez les calculer spécifiquement pour ce sous-ensemble de poissons et de chats, mais ces valeurs sont suffisamment décentes. 
Si vous travailliez sur un ensemble de données complètement différent, vous devriez calculer cette moyenne et cet écart, bien que de nombreuses personnes utilisent simplement ces constantes ImageNet et rapportent des résultats acceptables.

## Construire des Données de Test et de Validation

Nous téléchargeons un ensemble de validation, qui est une série d'images de chats et de poissons qui n'apparaissent pas dans l'ensemble d'entraînemnt. À la fin de chaque cycle d'entraînemnt (également appelé *epoch*), nous comparons cet ensemble pour nous assurer que notre réseau ne fait pas d'erreur.

In [ ]:
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(
    root=val_data_path,
    transform=transforms
)

En plus d'un ensemble de validation, nous devons également créer un ensemble de test. Celui-ci est utilisé pour tester le modèle une fois l'entraînement terminé :

In [ ]:
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(
    root=test_data_path,
    transform=transforms
)

Nous pouvons désormais contruire notre data loader en quelques lignes :

In [ ]:
batch_size = 64
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

Par défaut, les chargeurs de données de PyTorch sont réglés sur un `batch_size` de 1. Vous voudrez certainement changer cela. Bien que j'ai choisi 64 ici, vous pouvez expérimenter pour voir quelle taille de minibatch vous pouvez utiliser sans épuiser la mémoire de votre GPU. Vous pouvez également expérimenter avec certains des paramètres supplémentaires de PyTorch.

## Création d'un réseau

La création d'un réseau dans PyTorch est une affaire très Pythonique. Nous héritons d'une classe appelée `torch.nn.Network` et remplissons les méthodes `__init__` et `forward` :

In [ ]:
"""import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
"""
class SimpleNet(torch.nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = torch.nn.Linear(12288, 84)
        self.fc2 = torch.nn.Linear(84, 50)
        self.fc3 = torch.nn.Linear(50, 2)

    def forward_(self, x):
        x = x.view(-1, 12288)
        x = torch.nn.functional.relu(self.fc1(x))
        x = torch.nn.functional.relu(self.fc2(x))
        x = torch.nn.functional.softmax(self.fc3(x))
        return x

In [ ]:
simplenet = SimpleNet()

Nous effectuons toute configuration nécessaire dans `init()`, dans ce cas, nous appelons notre constructeur **superclass** et les trois couches entièrement connectées (appelées `Linear` dans PyTorch, par opposition à `Dense` dans Keras). La méthode `forward_()` décrit comment les données circulent à travers le réseau à la fois pour l'entraînement et pour faire des prédictions (inférence). Tout d'abord, nous devons convertir le tenseur 3D ($x$ et $y$ plus les informations de couleur à trois canaux - rouge, vert, bleu -) en un tenseur 1D afin qu'il puisse être introduit dans la première couche `Linear`, et nous le faisons en utilisant `view()`. A partir de là, vous pouvez voir que nous appliquons les fonctions d'activation des couches dans l'ordre, pour finalement retourner la sortie `softmax` et nous donner notre prédiction pour cette image.

Les nombres dans les couches cachées sont quelque peu arbitraires, à l'exception de la sortie de la dernière couche, qui est 2, correspondant à nos deux classes. En général, vous voulez que les données de vos couches soient compressées au fur et à mesure qu'elles descendent dans la pile. Si une couche va, disons, de 50 entrées à 100 sorties, alors le réseau pourrait apprendre en passant simplement les 50 connexions à 50 des 100 sorties et considérer que son travail est terminé. En réduisant la taille de la sortie par rapport à l'entrée avec moins de ressources, ce qui signifie, espérons-le, qu'il extrait certaines caractéristiques des images qui sont importantes pour le problème que nous essayons de résoudre ; par exemple, apprendre à repérer une nageoire ou une queue.

Nous avons une prédiction, et nous pouvons la comparer avec l'étiquette réelle de l'image originale pour voir si la prédiction était correcte. Mais nous avons besoin d'un moyen de permettre à PyTorch de quantifier non seulement si une prédiction est juste ou fausse, mais aussi à quel point elle est juste ou fausse. Ceci est géré par une fonction de perte (*loss function*).

## Fonctions de Perte (*Loss Functions*)

PyTorch est livré avec une collection complète qui couvre la plupart des applications que vous êtes susceptible de rencontrer, et vous pouvez bien sûr écrire vos propres applications si vous avez un domaine très personnalisé. Dans notre cas, nous allons utiliser une fonction de perte intégrée appelée `CrossEntropyLoss` qui est recommandée pour les tâches de catégorisation multiclasse comme celle que nous effectuons ici. Une autre fonction de perte que vous êtes susceptible de rencontrer est `MSELoss`, qui est la perte quadratique moyenne standard que vous pourriez utiliser lorsque vous faites une prédiction numérique.

Une chose à laquelle il faut faire attention avec `CrossEntropyLoss` est qu'il incorpore également `softmax()` dans le cadre de ses opérations, ainsi notre méthode `forward_()` devient la suivante :

In [ ]:
%%add_to SimpleNet

def forward(self, x):
    x = x.view(-1, 12288)
    x = torch.nn.functional.relu(self.fc1(x))
    x = torch.nn.functional.relu(self.fc2(x))
    x = self.fc3(x)
    return x

## Optimisation

Pour le problème des minima locaux, nous apportons une légère modification au fait que nous prenons tous les gradients possibles et indiquons des gradients aléatoires pendant un batch. Connue sous le nom de [*stochastic gradient descent* (SGD)](https://www.geeksforgeeks.org/ml-stochastic-gradient-descent-sgd/)~[[2]](#2), il s'agit de l'approche traditionnelle de l'optimisation des réseaux neuronaux et autres techniques d'apprentissage automatique. Mais d'autres optimiseurs sont disponibles, et en fait pour l'apprentissage profond, préférables. PyTorch est livré avec SGD et d'autres optimiseurs comme [AdaGrad](https://www.geeksforgeeks.org/intuition-behind-adagrad-optimizer/) et [RMSProp](https://towardsdatascience.com/understanding-rmsprop-faster-neural-network-learning-62e116fcf29a), ainsi qu'[Adam](https://www.geeksforgeeks.org/intuition-of-adam-optimizer/)~[[3]](#3).

L'une des principales améliorations apportées par **Adam** (tout comme **RMSProp** et **AdaGrad**) est qu'il utilise un taux d'apprentissage par paramètre, et adapte ce taux d'apprentissage en fonction du taux de changement de ces paramètres. Il conserve une liste exponentiellement décroissante de gradients et du carré de ces gradients et les utilise pour mettre à l'échelle le taux d'apprentissage global avec lequel **Adam** travaille. Il a été démontré empiriquement que **Adam** surpasse la plupart des autres optimiseurs dans les réseaux d'apprentissage profond, mais vous pouvez remplacer Adam par **SGD** ou **RMSProp** ou un autre optimiseur pour voir si l'utilisation d'une technique différente permet un apprentissage plus rapide et plus efficace pour votre application particulière.

La création d'un optimiseur basé sur **Adam** est simple. Nous appelons `torch.optim.Adam()` et passons les poids du réseau qu'il va mettre à jour (obtenus via `simmplenet.parameters()`) et notre taux d'apprentissage de 0.001 :

In [ ]:
optimizer = torch.optim.Adam(simplenet.parameters(), lr=0.001)

## Entraînement

In [ ]:
for epoch in range(epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        imput, target = batch
        output = model(input)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()

Nous prenons un batch de notre ensemble d'entraînement à chaque itération de la boucle, ce qui est géré par notre data loader. Nous les soumettons ensuite à notre modèle et calculons la perte à partir de la sortie attendue. Pour calculer les gradients, nous appelons la méthode `backward()` sur le modèle. La méthode `optimizer.step()` utilise ensuite ces gradients pour effectuer l'ajustement des poids dont nous avons parlé dans la section précédente.

Mais à quoi sert cet appel `zero_grad()` ? Il s'avère que les gradients calculés s'accumulent par défaut, ce qui signifie que si nous ne mettons pas à zéro les gradients à la fin de l'itération du batch, le batch suivant devra gérer le gradient de ce batch ainsi que le sien, et le batch suivant devra gérer les deux précédents, et ainsi de suite. Cela n'est pas utile, car nous voulons examiner uniquement les gradients du batch actuel pour notre optimisation à chaque itération. Nous utilisons `zero_grad()` pour nous assurer qu'ils sont remis à zéro une fois que nous avons terminé notre boucle.

## Fonctionnement sur GPU

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
else:
    device = torch.device("cpu")

simplenet.to(device)

Ici, nous copions le modèle sur le GPU si PyTorch signale qu'un GPU est disponible, ou sinon nous gardons le modèle sur le CPU. 

## Mise en Commun

In [ ]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, target = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)

        training_loss /= len(train_loader.dataset)
        model.eval()
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [ ]:
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, epochs=5, device=device)

## Prédictions

Voici un petit bout de code Python qui chargera une image depuis le système de fichiers et dira si notre réseau prédit un chat ou poisson.

In [ ]:
from PIL import Image

labels = ["chat","poisson"]

img = Image.open("./val/fish/100_1422.JPG") 
img = img_transforms(img).to(device)
img = torch.unsqueeze(img, 0)

simplenet.eval()
prediction = F.softmax(simplenet(img), dim=1)
prediction = prediction.argmax()
print(labels[prediction])

Comme notre réseau utilise des lots, il s'attend en fait à un tenseur 4D, la première dimension représentant les différentes images d'un lot. Nous n'avons pas de lot, mais nous pouvons créer un lot de longueur 1 en utilisant `unsqueeze(0)`, qui ajoute une nouvelle dimension à l'avant de notre tenseur.

## Sauvegarder le Modèle

Si vous êtes satisfait des performances d'un modèle ou si vous avez besoin de vous arrêter, vous pouvez sauvegarder l'état actuel d'un modèle au format *pickle* de Python en utilisant la méthode `torch.save()`. Inversement, vous pouvez charger une interation précédemment sauvegardée d'un modèle en utilisant la méthode `torch.load()`.

In [ ]:
torch.save(simplenet, "/tmp/simplenet") 
simplenet = torch.load("/tmp/simplenet")

Cela permet de stocker les paramètres et la structure du modèle dans un fichier. Cela peut poser un problème si vous modifiez la structure du modèle ultérieurement. Pour cette raison, il est plus courant de sauvegarder le `state_dict` d'un modèle à la place. C'est un `dict` Python standard qui contient les cartes pour les paramètres de chaque couche du modèle.

In [ ]:
torch.save(simplenet.state_dict(), "/tmp/simplenet")    
simplenet = SimpleNet()
simplenet_state_dict = torch.load("/tmp/simplenet")
simplenet.load_state_dict(simplenet_state_dict)

# Réseaux Neuronaux Convolutifs

Avec des réseaux entièrement connectés, si vous essayez d'ajouter des couches supplémentaires ou d'augmenter considérablement le nombre de paramètres, vous allez certainement manquer de mémoire sur votre GPU. De plus, l'entraînement jusqu'à une précision décente prendrait un certain temps.

Il est vrai qu'un réseau entièrement connecté ou (*feed-forward*) peut fonctionner comme un approximateur universel, mais la théorie ne dit pas combien de temps il vous faudra pour l'entraîner à devenir cette approximation de la fonction que vous recherchez vraiment. Mais on peut faire mieux, surtout avec les images.

## Premier Modèle Convolutif

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

In [ ]:
class CNNNet(nn.Module):

    def __init__(self, num_classes=2):
        super(CNNNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

La première chose à remarquer est l'utilisation de `nn.Sequential()`. Cela nous permet de créer une chaîne de couches (*layers*). Lorsque nous utilisons une de ces chaînes dans `forward()`, l'entrée passe par chacune des couches en succession. Vous pouvez utiliser ceci pour décomposer votre modèle en arrangements plus logiques. Dans ce réseau, nous avons deux chaînes : le bloc `features` et le `classifier`. Jetons un coup d'oeil aux nouvelles couches que nous introduisons, en commençant par `Conv2d`.

## Convolutions

La couche `Conv2d` est une convolution 2D. Si nous avons une image en niveaux de gris, elle consiste en un tableau de $x$ pixels de large et $y$ pixels de haut, dont chaque entrée a une valeur qui indique si elle est noire ou blanche ou quelque part entre les deux (nous supposons une image de 8 bits, donc chaque valeur peut varier de 0 à 255).

![](https://1.cms.s81c.com/sites/default/files/2021-01-06/ICLH_Diagram_Batch_02_17A-ConvolutionalNeuralNetworks-WHITEBG.png)

# References

<a id="1">[1]</a> 
Rajat Raina et al. (2009). 
"Large-Scale Deep Unsupervised Learning Using Graphics Processors". 
*Proceedings of the 26th Annual International Conference on Machine Learning*, 873–880.
https://doi.org/10.1145/1553374.1553486

<a id="2">[2]</a> 
Sebastian Ruder (2016). 
"An overview of gradient descent optimization algorithms". 
*CoRR*, abs/1609.04747.
https://arxiv.org/abs/1609.04747

<a id="3">[3]</a> 
Diederik P.Kingama, Jimmy Ba (2014). 
"Adam: A Method for Stochastic Optimization". 
*3rd International Conference on Learning Representations, ICLR 2015*.
https://arxiv.org/abs/1412.6980